### Universidad Nacional de Lujan - Bases de Datos Masivas (11088) - Cavasin Nicolas #143501
# TP05-01 - Árboles de decisión

### Ejercicio 1:
A  partir  del  dataset  presentado  a  continuación,  y  teniendo  en  cuenta  las fórmulas de entropía y ganancia de información, calcule y diagrame el árbol de decisión que le permita decidir si comer asado o no en función del clima.

In [3]:
#!conda update --update-all -y
#!conda update -y conda
#!conda install -y pandas
#!conda install -y scikit-learn
#!conda install -y graphviz

In [5]:
import os.path

if os.path.exists("db-asado.xls"):
    !echo Archivo encontrado.
    !echo Eliminando db-asado.xls...
    !rm db-asado.xls

# Descargo el archivo con el cual se va a trabajar
!echo 
!echo Iniciando descarga...
!echo
!echo
!wget https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0501/db-asado.xls

Archivo encontrado.
Eliminando db-asado.xls...

Iniciando descarga...


--2020-11-11 17:10:59--  https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0501/db-asado.xls
Loaded CA certificate &#39;/etc/ssl/certs/ca-certificates.crt&#39;
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.216.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.216.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29184 (28K) [application/octet-stream]
Saving to: &#39;db-asado.xls&#39;

db-asado.xls        100%[===================&gt;]  28.50K  --.-KB/s    in 0.02s   

2020-11-11 17:10:59 (1.75 MB/s) - &#39;db-asado.xls&#39; saved [29184/29184]



In [7]:
# Importo pandas para leer el archivo
import pandas as pd

# Leo el archivo
datos = pd.read_excel("db-asado.xls")
datos.head()

,PRONÓSTICO,TEMPERATURA,HUMEDAD,VIENTO,ASADO
0,soleado,calor,alta,leve,no
1,soleado,calor,alta,fuerte,no
2,nublado,calor,alta,leve,si
3,lluvioso,templado,alta,leve,si
4,lluvioso,frío,normal,leve,si


In [8]:
# Muestro su distribución de tuplas y columnas
print(f"Total de tuplas = {datos.shape[0]}.")
print(f"Total de columnas = {datos.shape[1]}.")

Total de tuplas = 14.
Total de columnas = 5.


Como se puede observar, los datos almacenados son del tipo *String* y eso es un problema para la generación del árbol de clasificación. Por lo tanto, se deben transformar a valores numéricos y categóricos (discretos).

Durante la transformación, por cada **columna original** se *crearán* tantas **nuevas columnas** como **diferentes valores** posea cada columna y se representará su valor al estilo bitmap según la nueva columna a la que pertenezca, dejando las otras en 0. Este método es conocido como **one-hot-encoding**.

Para ser más claro, se ejemplifica a continuación con una de las 4 columnas de features (pero se aplica a todas).

In [9]:
# Importo el encoder necesario para transformar
from sklearn.preprocessing import OneHotEncoder 

# Guardo el target
target = datos.pop("ASADO")

# Guardo las features
features = datos.copy()

# Instancio el encoder
ohe = OneHotEncoder(sparse=False, handle_unknown='error')

# Le enseño a codificar las features y las guardo en un dataframe
trans_features = pd.DataFrame(ohe.fit_transform(features))
trans_features.head()
# Guardo los nombres
trans_feature_names = ohe.get_feature_names()

asdasd

In [8]:
# Importo la libreria para construir el arbol
from sklearn import tree

# Importo libreria para graficar 
import graphviz

# Importo un clasificador para separar set de entrenamiento y set de testeo
from sklearn.model_selection import train_test_split as splitter

# Separo en 80:20 entrenamiento y testeo
train_features, test_features, train_target, test_target = splitter(trans_features, target, random_state=0, test_size=0.3)

# Instancio el arbol para que sea clasificador y que lo haga por entropia
t = tree.DecisionTreeClassifier(criterion='entropy')

# Entreno el arbol
t.fit(train_features, train_target)

# Realizo las predicciones en función del árbol generado
predictions = t.predict(test_features)

# Exporto el arbol en formato DOT (ver referencias abajo) para usar con graphviz
dot_file = tree.export_graphviz(t 
                    , out_file=None
                    , feature_names=trans_feature_names
                    , class_names=target.name
                    , filled=True
                    , rounded=True
                    , special_characters=True
                    , precision=4)


# Capturo el source
graph = graphviz.Source(dot_file)

# Le digo que lo almacene como .png
graph.format = "png"
graph.save('asado')



&#39;asado&#39;

![Arbol](asado.png)

In [9]:
from sklearn import metrics 

# Vamos a testear el modelo
acc = metrics.accuracy_score(target, predictions)
print(f"Accuracy: {acc}")

# Vemos un reporte de clasificación de varias métricas
print(metrics.classification_report(target, predictions))

ValueError: Found input variables with inconsistent numbers of samples: [14, 5]

### Referencias:

- La [indexación][0] de data frames en Pandas. 
- Como funciona el [preprocesamiento][1] en Scikit-Learn.
- [Workflow][2] de transformaciones por el método 'one-hot-encoding'.
- Formato [DOT][3].

[0]:https://pandas.pydata.org/docs/user_guide/indexing.html#indexing
[1]:https://scikit-learn.org/stable/modules/preprocessing.html
[2]:https://medium.com/dunder-data/from-pandas-to-scikit-learn-a-new-exciting-workflow-e88e2271ef62 
[3]:https://graphviz.org/doc/info/lang.html